In [1]:
from sklearn import mixture
from MNE_Pipeline_ext import MNE_Repo_Mat_ext
from MNE_Pipeline import MNE_Repo_Mat as MP
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import pickle

In [2]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        5.2G        238M        421M         25G         25G
Swap:          2.0G         46M        2.0G


In [2]:
paths = np.sort([os.path.join('Data', file) for file in os.listdir('Data') if file.endswith('.mat')])
subjects = np.sort([os.path.splitext(file)[0] for file in os.listdir('Data/') if file.endswith('.mat')])

## Load Raw .mat data

In [3]:
mp_objs = dict()
mp = MP()
info = mp.construct_info()
del mp
RT_dict_sub = dict()

for path, subject in tqdm(zip(paths, subjects)):
    mp = MP()
    mp.load_data_mat(path)
    mp_objs[subject] = mp

/usr/local/lib/python3.6/dist-packages/mne/utils/docs.py:830: DeprecationWarning: Function read_montage is deprecated; ``read_montage`` is deprecated and will be removed in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_custom_montage``, or ``read_dig_captrack`` to read a digitization based on your needs instead; or ``make_standard_montage`` to create ``DigMontage`` based on template; or ``make_dig_montage`` to create a ``DigMontage`` out of np.arrays
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/mne/utils/docs.py:813: DeprecationWarning: Class Montage is deprecated; Montage class is deprecated and will be removed in v0.20. Please use DigMontage instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/cp_project_new/MNE_Pipeline.py:58: DeprecationWarning: Setting a montage using anything rather than DigMontage is deprecated and will raise an error in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_dig_polhemu

In [4]:
RT_dict_sub = pickle.load(open('RT_dict_sub', 'rb'))
RT_dict_sub

{'N1': array([ 660.,  396.,  418.,  470.,  380.,  386.,  370.,  436.,  320.,
         376.,  390.,  330.,  340.,  322.,  334.,  348.,  320.,  416.,
         300.,  356.,  694.,  400.,  350.,  392.,  400.,  366.,  450.,
         364.,  344.,  380.,  356.,  352.,  312.,  384.,  278.,  392.,
         370.,  750.,  522.,  374.,  380.,  374.,  440.,  466.,  474.,
         314.,  436.,  480.,  510.,  350.,  690.,  724.,  406.,  506.,
         470.,  408.,  302.,  384.,  730.,  366.,  368.,  360.,  350.,
         356.,  408.,  400.,  410.,  392.,  304.,  372.,  312.,  488.,
         390.,  512.,  398.,  328.,  366.,  358.,  246.,  330.,  470.,
         474.,  552.,  338.,  458.,  724.,  510.,  400.,  416.,  436.,
         460.,  510.,  432.,  410.,  612.,  350.,  590.,  936.,  434.,
         772.,  346.,  792.,  792.,  362.,  672.,  352.,  588.,  302.,
         754.,  284.,  478.,  280.,  352.,  296.,  348.,  654.,  526.,
         352.,  460.,  408.,  394.,  336.,  262.,  256.,  386.,  442.,


## Filter RTs and Remove Trials with 'nan' or < 0 RTs

In [4]:
for subject in tqdm(subjects):
    RT = mp_objs[subject].RT
    epochs_raw = mp_objs[subject].epochs_raw
    
    index_invalid_RT = [i for i in range(RT.shape[0]) if RT[i] <= 0 or str(RT[i]) == 'nan']
    
    
    print('{}: {} RTs: {}, invalid_index: {}'.format(subject, mp_objs[subject].epochs_raw.shape,
                                                     mp_objs[subject].RT.shape, index_invalid_RT))
    
    mp_objs[subject].RT = np.delete(RT, index_invalid_RT)
    mp_objs[subject].epochs_raw = np.delete(epochs_raw, index_invalid_RT, axis=0)
    
    print('after {}: {} RTs: {}'.format(subject, mp_objs[subject].epochs_raw.shape, mp_objs[subject].RT.shape))

N1: (750, 64, 500) RTs: (750,), invalid_index: [525]
after N1: (749, 64, 500) RTs: (749,)
N10: (750, 64, 500) RTs: (750,), invalid_index: [586]
after N10: (749, 64, 500) RTs: (749,)
N11: (750, 64, 500) RTs: (750,), invalid_index: []
after N11: (750, 64, 500) RTs: (750,)
N12: (750, 64, 500) RTs: (750,), invalid_index: [702]
after N12: (749, 64, 500) RTs: (749,)
N13: (750, 64, 500) RTs: (750,), invalid_index: [78, 156, 188, 652]
after N13: (746, 64, 500) RTs: (746,)
N14: (750, 64, 500) RTs: (750,), invalid_index: [418, 529, 543, 564]
after N14: (746, 64, 500) RTs: (746,)
N15: (750, 64, 500) RTs: (750,), invalid_index: []
after N15: (750, 64, 500) RTs: (750,)
N2: (760, 64, 500) RTs: (760,), invalid_index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
after N2: (750, 64, 500) RTs: (750,)
N3: (750, 64, 500) RTs: (750,), invalid_index: []
after N3: (750, 64, 500) RTs: (750,)
N4: (750, 64, 500) RTs: (750,), invalid_index: []
after N4: (750, 64, 500) RTs: (750,)
N5: (750, 64, 500) RTs: (750,), invalid_index:

## Predict using Bayesian GMM and add new key
### remember 0-slow, 1-fast, 2-outlier, 3-medium

In [5]:
BNPY = pickle.load(open('Bayesian_GMM_RT_model.pkl', 'rb'))
BNPY

BayesianGaussianMixture(covariance_prior=None, covariance_type='spherical',
                        degrees_of_freedom_prior=None, init_params='random',
                        max_iter=5000, mean_precision_prior=0.8,
                        mean_prior=None, n_components=4, n_init=1,
                        random_state=42, reg_covar=0, tol=0.001, verbose=1,
                        verbose_interval=10, warm_start=False,
                        weight_concentration_prior=None,
                        weight_concentration_prior_type='dirichlet_distribution')

##### Add key to mp_objs.data_mat

In [6]:
for subject in tqdm(subjects):
    RTs = mp_objs[subject].RT
    RTs = RTs.reshape((len(RTs), 1))
    RT_clusters = BNPY.predict(RTs)
    mp_objs[subject].RT_clusters = RT_clusters

In [7]:
for subject in tqdm(subjects):
    mp_objs[subject].construct_info()

/usr/local/lib/python3.6/dist-packages/mne/utils/docs.py:830: DeprecationWarning: Function read_montage is deprecated; ``read_montage`` is deprecated and will be removed in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_custom_montage``, or ``read_dig_captrack`` to read a digitization based on your needs instead; or ``make_standard_montage`` to create ``DigMontage`` based on template; or ``make_dig_montage`` to create a ``DigMontage`` out of np.arrays
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/mne/utils/docs.py:813: DeprecationWarning: Class Montage is deprecated; Montage class is deprecated and will be removed in v0.20. Please use DigMontage instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/cp_project_new/MNE_Pipeline.py:58: DeprecationWarning: Setting a montage using anything rather than DigMontage is deprecated and will raise an error in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_dig_polhemu

/usr/local/lib/python3.6/dist-packages/mne/utils/docs.py:830: DeprecationWarning: Function read_montage is deprecated; ``read_montage`` is deprecated and will be removed in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_custom_montage``, or ``read_dig_captrack`` to read a digitization based on your needs instead; or ``make_standard_montage`` to create ``DigMontage`` based on template; or ``make_dig_montage`` to create a ``DigMontage`` out of np.arrays
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/mne/utils/docs.py:813: DeprecationWarning: Class Montage is deprecated; Montage class is deprecated and will be removed in v0.20. Please use DigMontage instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/cp_project_new/MNE_Pipeline.py:58: DeprecationWarning: Setting a montage using anything rather than DigMontage is deprecated and will raise an error in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_dig_polhemu

In [8]:
for subject in tqdm(subjects):
    RT_clusters = mp_objs[subject].RT_clusters
    events = mp_objs[subject].construct_events(RT_clusters)
    mp_objs[subject].construct_epoch_array(-0.2, events)

749 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: mean)
749 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: mean)
750 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: mean)
749 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: mean)
746 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: mean)
746 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Applying baseline corr

No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: mean)
999 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Applying baseline correction (mode: mean)



In [9]:
# Save Epochs
for subject in subjects:
    epochs = mp_objs[subject].epochs
    mp_objs[subject].save_epochs(epochs, subject)

/home/cp_project_new/MNE_Pipeline.py:80: RuntimeWarning: This filename (RT_epochs_bts_100/N1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_path_to_save, overwrite=True)
/home/cp_project_new/MNE_Pipeline.py:80: RuntimeWarning: This filename (RT_epochs_bts_100/N10.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_path_to_save, overwrite=True)
/home/cp_project_new/MNE_Pipeline.py:80: RuntimeWarning: This filename (RT_epochs_bts_100/N11.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_path_to_save, overwrite=True)
/home/cp_project_new/MNE_Pipeline.py:80: RuntimeWarning: This filename (RT_epochs_bts_100/N12.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.f

/home/cp_project_new/MNE_Pipeline.py:80: RuntimeWarning: This filename (RT_epochs_bts_100/S5.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_path_to_save, overwrite=True)
/home/cp_project_new/MNE_Pipeline.py:80: RuntimeWarning: This filename (RT_epochs_bts_100/S6.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_path_to_save, overwrite=True)
/home/cp_project_new/MNE_Pipeline.py:80: RuntimeWarning: This filename (RT_epochs_bts_100/S7.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(epoch_path_to_save, overwrite=True)
/home/cp_project_new/MNE_Pipeline.py:80: RuntimeWarning: This filename (RT_epochs_bts_100/S8.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif,

## Bootstrap with RT as Events

In [4]:
epoch_paths = np.sort([os.path.join('RT_epochs', epoch_file) for epoch_file in os.listdir('RT_epochs')])
mp_ext = MNE_Repo_Mat_ext()

sub_epochs = dict()

for path, subject in tqdm(zip(epoch_paths, subjects)):
    epochs = mp_ext.load_epochs(path)
    sub_epochs[subject] = epochs

/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/N1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/N10.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/N11.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/N12.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.f

/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/S6.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/S7.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/S8.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/S9.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.

## Save RT epochs for temporal modeling

In [4]:
epoch_paths = np.sort([os.path.join('RT_epochs', epoch_file) for epoch_file in os.listdir('RT_epochs')])
mp_ext = MNE_Repo_Mat_ext()

events = ['0', '1', '3']
subs = []
data = []
labels = []
for i, (path, sub) in tqdm(enumerate(zip(epoch_paths, subjects))):
    epochs = mp_ext.load_epochs(path)
    for event in events:
        data_sub = epochs[event].get_data()
        if i == 0 and int(event) == 0:
            data = data_sub
            subs.extend([sub] * len(data_sub))
            labels.extend([int(event)] * len(data_sub))
            continue
        subs.extend([sub] * len(data_sub))
        labels.extend([int(event)] * len(data_sub))
        data = np.append(data, data_sub, axis=0)
        

/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/N1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/N10.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/N11.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/N12.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.f

/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/S6.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/S7.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/S8.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  self.epochs = mne.read_epochs(path, verbose=0)
/home/kam/CP Project/MNE_Pipeline.py:83: RuntimeWarning: This filename (RT_epochs/S9.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.

In [9]:
np.save('data_st_RT.npy', data)
np.save('label_st_RT.npy', labels)
np.save('subject_st_RT.npy', subs)

In [6]:
slow = 0
med = 0
fast = 0
for sub in sub_epochs:
         
    slow = slow + len(sub_epochs[sub]['0'])
    med = med + len(sub_epochs[sub]['3'])
    fast = fast + len(sub_epochs[sub]['1'])
    
slow, med, fast

(4025, 11029, 30496)

In [5]:
4025 + 11029 + 30496

45550

In [5]:
for subject in tqdm(subjects):
    epochs = sub_epochs[subject]
    mp_ext.async_bootstrap_epochs_with_RT(subject, epochs, ['0','1', '3'], 
                                          sampling_rate=10, iterations=500, save_path='bootstrap_erps_RT_rate_10')

In [6]:
paths = [os.path.join('bootstrap_erps_RT_rate_10', sub) for sub in subjects]
slow = 0
med = 0
fast = 0
for path in paths:
    data = pickle.load(open(path, 'rb'))
    slow = len(data['0']) + slow
    med = len(data['3']) + med
    fast = len(data['1']) + fast

In [7]:
slow, med, fast

(24002, 25000, 25000)

In [3]:
erp_load_paths = np.sort([os.path.join('bootstrap_erps_RT_rate_10', subject) for subject in subjects])
slow = np.zeros((24002, 64, 500))
med = np.zeros((25000, 64, 500))
fast = np.zeros((25000, 64, 500))

subs_slow = []
subs_med = []
subs_fast = []
n_slow = 0
n_med = 0
n_fast = 0
for path, subject in tqdm(zip(erp_load_paths, subjects)):
    file = open(path, 'rb')
    erps = pickle.load(file)
    len_slow = len(erps['0'])
    len_med = len(erps['3'])
    len_fast = len(erps['1'])
    
#     t_len = len_slow + len_med + len_fast
    subs_slow.extend([subject] * len_slow)
    subs_med.extend([subject] * len_med)
    subs_fast.extend([subject] * len_fast)
    
    slow[n_slow : n_slow + len_slow] = erps['0']
    med[n_med : n_med + len_med] = erps['3']
    fast[n_fast : n_fast + len_fast] = erps['1']
    
    n_slow = n_slow + len_slow
    n_med = n_med + len_med
    n_fast = n_fast + len_fast
    file.close()

In [4]:
subs_slow = np.array(subs_slow).reshape((-1,1))
subs_med = np.array(subs_med).reshape((-1,1))
subs_fast = np.array(subs_fast).reshape((-1,1))

In [5]:
lbl_slow = np.full((n_slow, 1), 0)
lbl_med = np.full((n_med, 1), 3)
lbl_fast = np.full((n_fast, 1), 1)

labels = np.vstack((lbl_slow, lbl_med, lbl_fast))

subs = np.vstack((subs_slow, subs_med, subs_fast))

labels = pd.DataFrame(labels)
subs = pd.DataFrame(subs)

labels.insert(0, 'trial_no', [i+1 for i in range(len(labels))])
subs.insert(0, 'trial_no', [i+1 for i in range(len(subs))])

labels.columns = ['trial_no', 'labels']

In [6]:
labels.to_csv("labels_RT_10.csv")
subs.to_csv("subs_RT_10.csv")

In [8]:
trial_no = 1

for i, data in tqdm(enumerate(slow)):
    path = os.path.join("RT_10_data", "{}.npy".format(trial_no))
    np.save(path, data)
    trial_no += 1
    
for i, data in tqdm(enumerate(med)):
    path = os.path.join("RT_10_data", "{}.npy".format(trial_no))
    np.save(path, data)
    trial_no += 1
    
for i, data in tqdm(enumerate(fast)):
    path = os.path.join("RT_10_data", "{}.npy".format(trial_no))
    np.save(path, data)
    trial_no += 1

## GET Avg Band Power from ERPs

In [2]:
mp = MP()
mp.Fs = 500
info = mp.construct_info()

/home/kam/anaconda3/envs/mne/lib/python3.7/site-packages/mne/utils/docs.py:830: DeprecationWarning: Function read_montage is deprecated; ``read_montage`` is deprecated and will be removed in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_custom_montage``, or ``read_dig_captrack`` to read a digitization based on your needs instead; or ``make_standard_montage`` to create ``DigMontage`` based on template; or ``make_dig_montage`` to create a ``DigMontage`` out of np.arrays
  warnings.warn(msg, category=DeprecationWarning)
/home/kam/anaconda3/envs/mne/lib/python3.7/site-packages/mne/utils/docs.py:813: DeprecationWarning: Class Montage is deprecated; Montage class is deprecated and will be removed in v0.20. Please use DigMontage instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/kam/CP Project/MNE_Pipeline.py:58: DeprecationWarning: Setting a montage using anything rather than DigMontage is deprecated and will raise an error in v0.20. Please use ``read_dig_fif``, 

In [5]:
erp_load_paths = np.sort([os.path.join('bootstrap_erps_RT_rate_50_1000', subject) for subject in subjects])
sub_bps = dict()
for path, subject in tqdm(zip(erp_load_paths, subjects)):
    file = open(path, 'rb')
    erps = pickle.load(file)
    file.close()
    bp_dict = dict()
    for event_id in erps:
        mp.epochs_raw = erps[event_id]
        bp = mp.get_avg_band_power(tmin=-0.2, tmax=0.8)
        bp_dict[event_id] = bp
        mp.epochs_raw = None
    sub_bps[subject] = bp_dict

In [6]:
total_slow = 0
total_fast = 0
total_medium = 0
for subject in subjects:
    total_slow = total_slow + len(sub_bps[subject]['0'])
    total_fast = total_fast + len(sub_bps[subject]['1'])
    total_medium = total_medium + len(sub_bps[subject]['3'])
print("total number of examples in slow class: {}".format(total_slow))
print("total number of examples in fast class: {}".format(total_fast))
print("total number of examples in medium class: {}".format(total_medium))
print("total number of examples: {}".format(total_slow + total_fast + total_medium))

total number of examples in slow class: 26024
total number of examples in fast class: 50000
total number of examples in medium class: 49001
total number of examples: 125025


In [6]:
mp = MP()
mp.Fs = 500
info = mp.construct_info()

/usr/local/lib/python3.6/dist-packages/mne/utils/docs.py:830: DeprecationWarning: Function read_montage is deprecated; ``read_montage`` is deprecated and will be removed in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_custom_montage``, or ``read_dig_captrack`` to read a digitization based on your needs instead; or ``make_standard_montage`` to create ``DigMontage`` based on template; or ``make_dig_montage`` to create a ``DigMontage`` out of np.arrays
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/mne/utils/docs.py:813: DeprecationWarning: Class Montage is deprecated; Montage class is deprecated and will be removed in v0.20. Please use DigMontage instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/cp_project_new/MNE_Pipeline.py:58: DeprecationWarning: Setting a montage using anything rather than DigMontage is deprecated and will raise an error in v0.20. Please use ``read_dig_fif``, ``read_dig_egi``, ``read_dig_polhemu

In [7]:
sub_bps = dict()
for subject in tqdm(subjects):
    erps_dict = sub_erps[subject]
    bp_dict = dict()
    for event_id in erps_dict:
        mp.epochs_raw = erps_dict[event_id]
        bp = mp.get_avg_band_power(tmin=-0.2, tmax=0.8)
        bp_dict[event_id] = bp
        mp.epochs_raw = None
    sub_bps[subject] = bp_dict

In [7]:
pickle.dump(sub_bps, open('avg_bp_dict_RT_50_1000.pkl', 'wb'))

In [3]:
sub_bps = pickle.load(open('avg_bp_dict_RT_50_1000.pkl', 'rb'))

labels = []
data = []
subs = []
for i, subject in tqdm(enumerate(subjects)):
    bp_dict = sub_bps[subject]
    for j, event_id in enumerate(bp_dict):
        no_samples = len(bp_dict[event_id])
        lbls = np.full((no_samples, ), int(event_id))
        labels = np.append(labels, lbls)
        _subs = np.full((no_samples, ), subject)
        avg_bps = bp_dict[event_id]
        subs = np.append(subs, _subs)
        
        if i == 0 and j == 0:
            data.append(avg_bps)
            data = np.array(data).reshape((-1, 192))
            print(data.shape)
            continue
            
        data = np.append(data, avg_bps, axis=0)
        
np.savetxt('data_bp_RT_50_1000.csv', data, delimiter=',')
np.savetxt('labels_RT_50_1000.csv', labels, delimiter=',')
np.savetxt('subjects_RT_50_1000.csv', subs, delimiter=',', fmt='%s')

(1, 192)



In [1]:
data.shape

NameError: name 'data' is not defined